In [2]:
import scipy.misc
import numpy as np
from sklearn.linear_model import LinearRegression
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import pyplot, transforms
import math

image interpolation?
threshold


In [ ]:
def linreg(x1,y1):
    model = LinearRegression()
    x1 = x1.reshape(-1, 1)
    y1 = y1.reshape(-1, 1)
    model.fit(x1, y1)
    print(f"intercept: {model.intercept_}")
    print(f"slope: {model.coef_}")
    plt.scatter(x1, y1,color='g')
    plt.plot(x1, model.predict(x1),color='k')

In [ ]:

def fractal_dimension(Z, threshold=0.9):

    # Only for 2d image
    #print((Z.shape))
    assert(len(Z.shape) == 2)

    
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])


    # Transform Z into a binary array
    Z = (Z <= threshold)

    # Minimal dimension of image
    p = min(Z.shape)
    
    #print(Z.shape)
    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))
    #print(n)
    
    # Extract the exponent
    n = int(np.log(n)/np.log(2))
    #print(n)
    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)
    #print(sizes)
    #print(sizes)
    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    x = np.log(sizes)
    y = np.log(counts)
    
    return -coeffs[0], x, y


#coeff, x, y  = fractal_dimension(I)
#print("Minkowski–Bouligand dimension (computed): ", coeff)


In [ ]:
def start(filename, pixel_threshold = 0.9):
    I = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    
    # testing inverse of inages
    #I = cv2.bitwise_not(I)
    #
    I = I/256
    
    
    coeff, x, y  = fractal_dimension(I, pixel_threshold)
    #print("Minkowski–Bouligand dimension (computed): ", coeff)
    return coeff, x, y

In [ ]:
def save_csv(xs, ys, filename):
    data_dict = []
    for i in  range(len(xs)):
        xval = str(xs[i])
        yval = str(ys[i])
        temp = dict(xs = xval, ys = yval)
        data_dict.append(temp)

    field_names = ['xs', 'ys']
    with open(filename, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = field_names)
        writer.writeheader()
        writer.writerows(data_dict)

In [131]:
ranges ={'Violet': [(150, 100, 100), (180, 255, 255)],
 'Indigo': [(135, 100, 100), (150, 255, 255)],
 'Blue': [(90, 100, 100), (135, 255, 255)],
 'Green': [(35, 100, 100), (90, 255, 255)],
 'Yellow': [(25, 100, 100), (35, 255, 255)],
 'Orange': [(10, 100, 100), (25, 255, 255)],
 'Red': [(0,100,100),(10,100,100)]}

In [132]:
def remove_colour(img, fname, col_min, col_max):
    upstate = cv2.imread(img)
    upstate_hsv = cv2.cvtColor(upstate, cv2.COLOR_BGR2HSV)

    mask_inverse = cv2.inRange(upstate_hsv, col_min, col_max)
    mask = mask_inverse #cv2.bitwise_not(mask_inverse)
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)    
    masked_upstate = cv2.bitwise_and(upstate, mask_rgb)
    
    gray = cv2.cvtColor(masked_upstate, cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    masked_upstate[thresh == 255] = 0
    
    cv2.imwrite(fname, masked_upstate)
    
    
    

In [133]:

def remove_red(img, fname):
    image = cv2.imread(img)
    result = image.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2HSV)
    lower1 = np.array([0, 100, 20])
    upper1 = np.array([10, 255, 255])
    lower2 = np.array([160,100,20])
    upper2 = np.array([179,255,255])

    lower_mask = cv2.inRange(result, lower1, upper1)
    upper_mask = cv2.inRange(result, lower2, upper2)

    full_mask = lower_mask + upper_mask;

    result = cv2.bitwise_and(result, result, mask=full_mask)
    upstate_hsv = cv2.cvtColor(result, cv2.COLOR_BGR2HSV)
    mask_rgb = upstate_hsv #cv2.bitwise_not(upstate_hsv)
    masked_upstate = cv2.bitwise_and(image, mask_rgb)
    
    gray = cv2.cvtColor(masked_upstate, cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    masked_upstate[thresh == 255] = 0
    
    
    cv2.imwrite(fname, masked_upstate)


In [134]:
def main_run(main_img):
    for i in ranges.keys():
        if i == "Red":
            remove_red('./GivenClusters/{}.png'.format(main_img), "./colours/{}/RedPrev.png".format(main_img))
        else:
            mini = np.array((ranges[i])[0], np.uint8)
            maxi = np.array((ranges[i])[1], np.uint8)
            remove_colour('./GivenClusters/{}.png'.format(main_img), "./colours/{}/{}.png".format(main_img, i), mini, maxi)

In [11]:
main_run("turnerimg")

In [135]:
main_run("ALE_a0_eta0_sigmad_n8000")

In [40]:

upstate1 = cv2.imread("./colours/turnerimg/RedEdited.png")
upstate1 = cv2.cvtColor(upstate1, cv2.COLOR_BGR2HSV)
upstate2 = cv2.imread("./colours/turnerimg/Orange32.png")
upstate2 = cv2.cvtColor(upstate2, cv2.COLOR_BGR2HSV)
masked_upstate = cv2.bitwise_and(upstate1, upstate2)
import cv2
import numpy as np

vis = np.concatenate((upstate1, upstate2), axis=0)
cv2.imwrite('out.png', vis)

True

In [62]:
def image_res(file_path , new_dims):
    ROOT = "./colours/"
    old_path = ROOT + file_path
    #new_path = ROOT + file_path[:-4]+ "adjusted"+ file_path[-4:]
    
    #dims = (cv2.imread(old_path, cv2.IMREAD_GRAYSCALE)).shape
    #dims = I.shape
    #print(dims)
    #new_dims = int(dims[0]*resolution), int(dims[1]*resolution)
    #new_dims = int(max(dims)*resolution) , int(max(dims)*resolution)
    #print(new_dims)
    
    im = Image.open(old_path)
    im_resized = im.resize(new_dims, Image.ANTIALIAS)
    return im_resized

In [121]:
def combine(col_list, fname):
    to_reduce = []
   
    red = cv2.imread("./colours/" + col_list[0].split('/')[0] + "/Red.png" , cv2.IMREAD_GRAYSCALE)
    red = red.shape
    print(red)
    #cv2.imwrite(red, "test.png")
    for col in col_list:
        curr_im = cv2.imread("./colours/"+col)#, cv2.IMREAD_GRAYSCALE)
        
        curr_im = cv2.resize(curr_im, red, Image.ANTIALIAS)
        to_reduce.append(curr_im)
    zones = np.maximum.reduce(to_reduce)
    alpha = np.any(zones>0, axis=2)
    alpha = Image.fromarray(alpha)
    alpha.save("./colours/" + col_list[0].split('/')[0]+fname)
#plt.imshow(alpha)

In [127]:
col_lst = ["turnerimg/Red.png","turnerimg/Orange.png", "turnerimg/Yellow.png","turnerimg/Green.png", "turnerimg/Blue.png", "turnerimg/Indigo.png", "turnerimg/Violet.png","turnerimg/RedPrev.png" ]

for i in range(len(col_lst)):

    count = i +1
    if count >= 8:
        combine(col_lst[1:], "/Gen8.png")
    combine(col_lst[:count], "/Gen{}.png".format(count))



(1920, 1920)
(1920, 1920)


/var/folders/jd/r79cklw57_bgyl4gtpf5f85c0000gn/T/ipykernel_26193/3179867084.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  curr_im = cv2.resize(curr_im, red, Image.ANTIALIAS)


(1920, 1920)
(1920, 1920)
(1920, 1920)
(1920, 1920)
(1920, 1920)
(1920, 1920)
(1920, 1920)


In [136]:
col_lst = ["ALE_a0_eta0_sigmad_n8000/Red.png","ALE_a0_eta0_sigmad_n8000/Orange.png", "ALE_a0_eta0_sigmad_n8000/Yellow.png","ALE_a0_eta0_sigmad_n8000/Green.png", "ALE_a0_eta0_sigmad_n8000/Blue.png", "ALE_a0_eta0_sigmad_n8000/Indigo.png", "ALE_a0_eta0_sigmad_n8000/Violet.png","ALE_a0_eta0_sigmad_n8000/RedPrev.png" ]

for i in range(len(col_lst)):

    count = i +1
    if count >= 8:
        combine(col_lst[1:], "/Gen8.png")
    combine(col_lst[:count], "/Gen{}.png".format(count))


(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)


/var/folders/jd/r79cklw57_bgyl4gtpf5f85c0000gn/T/ipykernel_26193/3179867084.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  curr_im = cv2.resize(curr_im, red, Image.ANTIALIAS)


(600, 600)


In [115]:
len(col_lst)

8